# Notebook to preprocess the data for the glider dashboard

Here we process/prepare the data files for:  
a) Sea surface fields  
b) Glider variables  
so they can be fed into the dashboard. 

In [ ]:
import numpy as np
import xarray as xr
import glidertools as gt

### Sea surface variables 

In the case of the SOGOS glider data we will use fields of sea surface height (and derived variables) and the finite scale Lyapunov exponents (FSLE). 

For many some glider experiments in cloud free regions other surface fields, like surface temperature or chlorophyll concentration, might also be available. 


In [ ]:
# Open data

# open Surface fields
# These were obtained from the Copernicus and Aviso websites,
# and manually cut for the time region and time period of the
# glider deployments. 
fname_SSH = "./data/SSH_sogos.nc"
fname_FSLE = "./data/FSLE_sogos.nc"

ds_ssh = xr.open_dataset(fname_SSH)
ds_fsle = xr.open_dataset(fname_FSLE)

In [ ]:
# Create variables for vectors
# these are the variables that are needed for the quiver plot function
ds_ssh['mag'] = np.sqrt(ds_ssh.ugos**2 + ds_ssh.vgos**2)
ds_ssh['angle'] = (np.pi/2.) - np.arctan2(ds_ssh.ugos/ds_ssh['mag'], 
                                          ds_ssh.vgos/ds_ssh['mag'])

In [ ]:
# Create an axis with time in YTD units, as it is easier to work with.
days = ds_ssh.time - np.datetime64('2019-01-01')
ds_ssh['days'] = (days / np.timedelta64(1, 'D'))

days = ds_fsle.time - np.datetime64('2019-01-01')
ds_fsle['days'] = (days / np.timedelta64(1, 'D'))

In [ ]:
del ds_ssh.attrs['_NCProperties']
# need to delete because of the issue:
# https://github.com/pydata/xarray/issues/2822

In [ ]:
# save to nc files 
ds_ssh.to_netcdf('./data/SSH_sogos_processed.nc', 'w')
ds_fsle.to_netcdf('./data/FSLE_sogos_processed.nc', 'w')

### Glider variables 

In [ ]:
# open glider files
glid_folder = './data/'
ds_CTD_659 = xr.load_dataset(glid_folder + 'sg659/CTD_659.nc')
ds_CTD_660 = xr.load_dataset(glid_folder + 'sg660/CTD_660.nc')

In [ ]:
ds_O2_659 = xr.load_dataset(glid_folder + 'sg659/O2_659.nc')
ds_O2_660 = xr.load_dataset(glid_folder + 'sg660/O2_660.nc')

ds_Chl_659 = xr.load_dataset(glid_folder + 'sg659/Chl_659.nc')
ds_Chl_660 = xr.load_dataset(glid_folder + 'sg660/Chl_660.nc')

#ds_BBP_659 = xr.load_dataset(glid_folder + 'sg659/BBP_659.nc')
#ds_BBP_660 = xr.load_dataset(glid_folder + 'sg660/BBP_660.nc')
# not working with at the moment because it is missing the time axis.

In [ ]:
# easier to work with a days variable that is a float rather than datenum
days = ds_CTD_659.time - np.datetime64('2019-01-01')
ds_CTD_659['days'] = (days / np.timedelta64(1, 'D'))

days = ds_O2_659.time - np.datetime64('2019-01-01')
ds_O2_659['days'] = (days / np.timedelta64(1, 'D'))

days = ds_Chl_659.time - np.datetime64('2019-01-01')
ds_Chl_659['days'] = (days / np.timedelta64(1, 'D'))

days = ds_CTD_660.time - np.datetime64('2019-01-01')
ds_CTD_660['days'] = (days / np.timedelta64(1, 'D'))

days = ds_O2_660.time - np.datetime64('2019-01-01')
ds_O2_660['days'] = (days / np.timedelta64(1, 'D'))

days = ds_Chl_660.time - np.datetime64('2019-01-01')
ds_Chl_660['days'] = (days / np.timedelta64(1, 'D'))

In [ ]:
# Calculate along track distance
dXdist = gt.utils.distance(ds_CTD_659.longitude, 
                           ds_CTD_659.latitude)/1e3 # Convert to km
ds_CTD_659['distance'] = xr.DataArray(np.nancumsum(dXdist), 
                                       dims=ds_CTD_659.dims,
                                       coords=ds_CTD_659.coords)

dXdist = gt.utils.distance(ds_CTD_660.longitude, 
                           ds_CTD_660.latitude)/1e3
ds_CTD_660['distance'] = xr.DataArray(np.nancumsum(dXdist), 
                                       dims=ds_CTD_660.dims,
                                       coords=ds_CTD_660.coords)


In [ ]:
ds_659_locs = xr.Dataset()
ds_660_locs = xr.Dataset()

# Group and average by dives so that plotting of positions is fast
ds_659_diveav = ds_CTD_659.groupby('dives').mean()
ds_660_diveav = ds_CTD_660.groupby('dives').mean()

ds_659_locs['longitude'] = ds_659_diveav.longitude
ds_659_locs['latitude']  = ds_659_diveav.latitude
ds_659_locs['days']      = ds_659_diveav.days
ds_659_locs['distance']  = ds_659_diveav.distance

ds_660_locs['longitude'] = ds_660_diveav.longitude
ds_660_locs['latitude']  = ds_660_diveav.latitude
ds_660_locs['days']      = ds_660_diveav.days
ds_660_locs['distance']  = ds_660_diveav.distance

In [ ]:
# We save the lat,lon, days and along track distance for each dive
ds_659_locs.to_netcdf('./data/659_locs.nc')
ds_660_locs.to_netcdf('./data/660_locs.nc')

In [ ]:
# Create some additional vatiable
ds_CTD_659['potdens'] = gt.physics.potential_density(ds_CTD_659.salinity, 
                                                 ds_CTD_659.temperature, 
                                                 ds_CTD_659.pressure, 
                                                 ds_CTD_659.latitude, 
                                                 ds_CTD_659.longitude)

ds_CTD_660['potdens'] = gt.physics.potential_density(ds_CTD_660.salinity, 
                                                 ds_CTD_660.temperature, 
                                                 ds_CTD_660.pressure,
                                                 ds_CTD_660.latitude, 
                                                 ds_CTD_660.longitude)

# we can add mixed layer depth, N2 etc in the future versions

In [ ]:
# Interpolate and grid glider data on pressure-time axis
# There are many ways this can be done. We choose a simple linear interpolation 
# in time and pressure.
# We could alternatively interpolate in density-time, pressure-distance, dive-pressure etc.
# This a place where a lot of more work into GP and 
# learning the most optimal interpolation (in sense of MLE) might work. 

# Note this is different from what glidertools does, which does a simple binning. 

# In future version we could also sort based on density field before applying 
# a bit of smoothing.

from scipy.interpolate import griddata
# interpolate on pressure-time 
def interp_pres_time(ds_glid, var): 
    pres_ug = ds_glid.pressure
    time_ug = ds_glid.days
    
    # convert to points values
    points = np.stack([time_ug.values, pres_ug.values],
                       axis=1)
    values = ds_glid[var].values
    
    # remove nans
    non_nan = np.logical_and(np.logical_and(~np.isnan(points[:,0]), 
                                      ~np.isnan(points[:,1])),
                                      ~np.isnan(values))
    
    points =points[non_nan,:]
    values =values[non_nan]
    
    # define grid
    pres_grid = np.linspace(0,1000,501)
    time_grid = np.arange(119, 207, 1/24)
    grid_p, grid_t = np.meshgrid(pres_grid, time_grid)
    
    temp_grided = griddata(points, values, 
                         (grid_t, grid_p), 
                         method='linear', rescale=True)
    
    return xr.DataArray(temp_grided.T, 
                               dims=["pressure", "time"],
                          coords={"pressure":pres_grid,
                                    "time":time_grid}).rename(var)

def interp_pres_dist(ds_glid, var): 
    pres_ug = ds_glid.pressure
    dist_ug = ds_glid.distance
    
    # convert to points values
    points = np.stack([dist_ug.values, pres_ug.values],
                       axis=1)
    values = ds_glid[var].values
    
    # remove nans
    non_nan = np.logical_and(np.logical_and(~np.isnan(points[:,0]), 
                                      ~np.isnan(points[:,1])),
                                      ~np.isnan(values))
    
    points =points[non_nan,:]
    values =values[non_nan]
    
    # define grid
    pres_grid = np.linspace(0,1000,501)
    dist_grid = np.arange(0, dist_ug.max().values, 1)
    grid_p, grid_d = np.meshgrid(pres_grid, dist_grid)
    
    temp_grided = griddata(points, values, 
                         (grid_d, grid_p), 
                         method='linear', rescale=True)
    
    return xr.DataArray(temp_grided.T, 
                               dims=["pressure", "distance"],
                          coords={"pressure":pres_grid,
                                    "distance":dist_grid}).rename(var)

# apply to all useful glider variables 
# can later add in variables measured by other instruments too 
def convert_glider_time_pres(ds_glid, vars_convert= ['temperature','salinity','potdens','spice']):
    
    #vars_convert = ['temperature','salinity','potdens']
    
    ds_grid = xr.Dataset()
    
    for v in vars_convert:
            ds_grid[v] = interp_pres_time(ds_glid, v)
    
    return ds_grid

def convert_glider_dist_pres(ds_glid, vars_convert= ['temperature','salinity','potdens','spice']):
    
    #vars_convert = ['temperature','salinity','potdens']
    
    ds_grid = xr.Dataset()
    
    for v in vars_convert:
            ds_grid[v] = interp_pres_dist(ds_glid, v)
    
    return ds_grid

In [ ]:
# method to go from O2 point -> CTD point
# Then can apply distance interpolation to O2 as well 
ds_O2_on_CTD_659 = xr.DataArray(np.interp(ds_CTD_659.days, ds_O2_659.days, ds_O2_659.oxygen),
                               dims = ds_CTD_659.dims, 
                               coords = ds_CTD_659.coords).rename('oxygen')
ds_Chl_on_CTD_659 = xr.DataArray(np.interp(ds_CTD_659.days, ds_Chl_659.days, ds_Chl_659.Chl),
                               dims = ds_CTD_659.dims, 
                               coords = ds_CTD_659.coords).rename('Chl')
ds_O2_on_CTD_660 = xr.DataArray(np.interp(ds_CTD_660.days, ds_O2_660.days, ds_O2_660.oxygen),
                               dims = ds_CTD_660.dims, 
                               coords = ds_CTD_660.coords).rename('oxygen')
ds_Chl_on_CTD_660 = xr.DataArray(np.interp(ds_CTD_660.days, ds_Chl_660.days, ds_Chl_660.Chl),
                               dims = ds_CTD_660.dims, 
                               coords = ds_CTD_660.coords).rename('Chl')

In [ ]:
ds_CTD_659['oxygen'] = ds_O2_on_CTD_659
ds_CTD_659['Chl'] = ds_Chl_on_CTD_659
ds_CTD_660['oxygen'] = ds_O2_on_CTD_660
ds_CTD_660['Chl'] = ds_Chl_on_CTD_660

In [ ]:
# convert from point data to gridded data, can take some time
ds_659_Tgrid = convert_glider_time_pres(ds_CTD_659, vars_convert= ['temperature','salinity','potdens', 'oxygen', 'Chl'])
ds_660_Tgrid = convert_glider_time_pres(ds_CTD_660, vars_convert= ['temperature','salinity','potdens', 'oxygen', 'Chl'])

ds_659_Dgrid = convert_glider_dist_pres(ds_CTD_659, vars_convert= ['temperature','salinity','potdens', 'oxygen', 'Chl'])
ds_660_Dgrid = convert_glider_dist_pres(ds_CTD_660, vars_convert= ['temperature','salinity','potdens', 'oxygen', 'Chl'])

In [ ]:
# Deprecated in favor of the above 2 step method
# Interpolate O2 and Chl to the same points
#ds_659_Tgrid = xr.merge([ds_659_Tgrid, interp_pres_time(ds_O2_659, 'oxygen')])
#ds_659_Tgrid = xr.merge([ds_659_Tgrid, interp_pres_time(ds_Chl_659, 'Chl')])

#ds_660_Tgrid = xr.merge([ds_660_Tgrid, interp_pres_time(ds_O2_660, 'oxygen')])
#ds_660_Tgrid = xr.merge([ds_660_Tgrid, interp_pres_time(ds_Chl_660, 'Chl')])

#ds_659_Dgrid = xr.merge([ds_659_Dgrid, interp_pres_dist(ds_O2_659, 'oxygen')])
#ds_659_Dgrid = xr.merge([ds_659_Dgrid, interp_pres_dist(ds_Chl_659, 'Chl')])

#ds_660_Dgrid = xr.merge([ds_660_Dgrid, interp_pres_dist(ds_O2_660, 'oxygen')])
#ds_660_Dgrid = xr.merge([ds_660_Dgrid, interp_pres_dist(ds_Chl_660, 'Chl')])

In [ ]:
# Estimate an anomaly field based on time mean 
# Could be defined in more complex ways too, like choose climatology as mean.
ds_659_Tgrid_anomaly = ds_659_Tgrid - ds_659_Tgrid.mean('time')
ds_660_Tgrid_anomaly = ds_660_Tgrid - ds_660_Tgrid.mean('time')

ds_659_Dgrid_anomaly = ds_659_Dgrid - ds_659_Dgrid.mean('distance')
ds_660_Dgrid_anomaly = ds_660_Dgrid - ds_660_Dgrid.mean('distance')

In [ ]:
# Estimate the distance axis that goes with the time
ds_659_Tgrid_loc = convert_glider_time_pres(ds_CTD_659, vars_convert= ['latitude','longitude'])
ds_660_Tgrid_loc = convert_glider_time_pres(ds_CTD_660, vars_convert= ['latitude','longitude'])

dXdist = gt.utils.distance(ds_659_Tgrid_loc.longitude.mean('pressure'), 
                           ds_659_Tgrid_loc.latitude.mean('pressure'))/1e3
ds_659_Tgrid['distance'] = np.nancumsum(dXdist)
ds_659_Tgrid_anomaly['distance'] = np.nancumsum(dXdist)

dXdist = gt.utils.distance(ds_660_Tgrid_loc.longitude.mean('pressure'), 
                           ds_660_Tgrid_loc.latitude.mean('pressure'))/1e3

ds_660_Tgrid['distance'] = np.nancumsum(dXdist)
ds_660_Tgrid_anomaly['distance'] = np.nancumsum(dXdist)

In [ ]:
# Estimate the time axis that goes with the distance 
temp = convert_glider_dist_pres(ds_CTD_659, vars_convert=['days'])
ds_659_Dgrid['time'] = temp.days.mean('pressure').values

temp = convert_glider_dist_pres(ds_CTD_660, vars_convert=['days'])
ds_660_Dgrid['time'] = temp.days.mean('pressure').values

In [ ]:
# Save the data
data_settings = {"zlib": True, "dtype":'float32', "complevel": 9}
encoding_dict = dict(temperature=data_settings,
                     salinity=data_settings,
                     potdens=data_settings,
                     oxygen=data_settings,
                     Chl=data_settings)



In [ ]:
ds_659_Tgrid.to_netcdf('./data/659_Tgrid.nc', encoding=encoding_dict)
ds_660_Tgrid.to_netcdf('./data/660_Tgrid.nc', encoding=encoding_dict)

ds_659_Tgrid_anomaly.to_netcdf('./data/659_Tgrid_anomaly.nc', encoding=encoding_dict)
ds_660_Tgrid_anomaly.to_netcdf('./data/660_Tgrid_anomaly.nc', encoding=encoding_dict)

In [ ]:
ds_659_Dgrid.to_netcdf('./data/659_Dgrid.nc', encoding=encoding_dict)
ds_660_Dgrid.to_netcdf('./data/660_Dgrid.nc', encoding=encoding_dict)

ds_659_Dgrid_anomaly.to_netcdf('./data/659_Dgrid_anomaly.nc', encoding=encoding_dict)
ds_660_Dgrid_anomaly.to_netcdf('./data/660_Dgrid_anomaly.nc', encoding=encoding_dict)